In [25]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager


In [26]:
 # Setup splinter INSTEAD
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/sabrinaalves/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


# NASA Mars News

In [27]:
#establish url of site
url = "https://redplanetscience.com/"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, "lxml")

In [28]:
#get first news title
news_title = soup.find_all("div", {"class": "content_title"})[0].text.strip()
news_title

'NASA Invites Public to Share Excitement of Mars 2020 Perseverance Rover Launch'

In [29]:
#get text of first news title
news_text= soup.find_all("div", {"class": "article_teaser_body"})[0].text.strip()
news_text

'There are lots of ways to participate in the historic event, which is targeted for July 30.'

# JPL Mars Space Images - Featured Image

In [6]:
url = "https://spaceimages-mars.com/"

browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, "lxml")

In [7]:
#get link of image in header
#inspiration for approach: https://www.codegrepper.com/code-examples/python/seautiful+soup+get+href+from+a+class
for image in soup.find_all("a", {"class": "showimg fancybox-thumbs"}):
    image_url = image["href"]
    
featured_image_url = url + image_url
featured_image_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

# Mars Facts

In [8]:
url = "https://galaxyfacts-mars.com/"

browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, "lxml")

In [9]:
#check to see how many tables there are
dfs = pd.read_html(html)
len(dfs)

2

In [10]:
#locate table for mars
mars_info = dfs[1]
mars_info

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [11]:
#rename columns
mars_info = mars_info.rename(columns={0:"category", 1:"mars_info"})
mars_info

,category,mars_info
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [12]:
#convert df to html file
mars_info.to_html("mars_info.html")

# Mars Hemispheres

In [13]:
url = "https://marshemispheres.com/"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, "lxml")

In [14]:
#review html text of webpage for code
items = soup.find_all("div", {"class": "item"})
item = items[0]
item

<div class="item">
<a class="itemLink product-item" href="cerberus.html"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a>
<div class="description">
<a class="itemLink product-item" href="cerberus.html">
<h3>Cerberus Hemisphere Enhanced</h3>
</a>
<span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/>
<p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p>
</div>
<!-- end description -->
</div>

In [15]:
#find url for image (click not working)
find_image = item.find("a")["href"]
image_link = url+ find_image

#rewrite into one line
image_link = url+item.find("a")["href"]
image_link

'https://marshemispheres.com/cerberus.html'

In [16]:
#let ChromeDriver travel to new page
url2 = image_link
browser.visit(url2)
html2 = browser.html
soup2 = BeautifulSoup(html2, "lxml")

In [17]:
#get title but drop the 'Enchanced' and spacing
title = soup2.find("h2", {"class":"title"}).text.strip('Enhanced').strip()
title

'Cerberus Hemisphere'

In [18]:
#get image url (does not work with URL 2)
image_item = soup2.find("img", {"class":"wide-image"})["src"]
img_url = url + image_item


#rewrite into one line
img_url = url + soup2.find("img", {"class":"wide-image"})["src"]
img_url

'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'

In [19]:
#above is testing for one item; return to first page
url = "https://marshemispheres.com/"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, "lxml")

In [20]:
#loop through page to get titles and image links for all 4 items

#list of dictionaries
hemisphere_image_urls = []


for item in items:
    #go to page when clicking on image
    image_link = url+item.find("a")["href"]
    url2 = image_link
    browser.visit(url2)
    html2 = browser.html
    soup2 = BeautifulSoup(html2, "lxml")
    
    #get title
    title = soup2.find("h2", {"class":"title"}).text.strip('Enhanced').strip()
    #get image url (full size)
    img_url = url + soup2.find("img", {"class":"wide-image"})["src"]
    
    #put into dictionary item
    hemisphere_image_urls.append({"title": title, "img_url": img_url})
    
    #return to starting page
    url = "https://marshemispheres.com/"
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, "lxml")

hemisphere_image_urls
    

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

# Moving to Flask App

In [23]:
#create a list of dictionaries of all items of all terms
#assistance in class for html item
mars_data = {}
mars_data["news_title"]=news_title
mars_data["news_text"]=news_text
mars_data["featured_image_url"]=featured_image_url
mars_data["mars_info"]=mars_info.to_html()
mars_data["hemisphere_image_urls"]=hemisphere_image_urls

mars_data

{'news_title': "Mars Helicopter Attached to NASA's Perseverance Rover",
 'news_text': "The team also fueled the rover's sky crane to get ready for this summer's history-making launch.",
 'featured_image_url': 'https://spaceimages-mars.com/image/featured/mars2.jpg',
 'mars_info': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>category</th>\n      <th>mars_info</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n 

In [ ]:
#browser.quit()